In [3]:
pip install tiledb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 9.5 MB/s  0:00:026m0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pysz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 2.7 MB/s  0:00:04m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2 

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    
    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.01)...
Internal SZ Ratio: 39.77
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 0.39 GB
Final Compression Ratio (rho): 39.7725
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.8495941162109375
Max Relative Error (calc): 0.009999874047935009
Target Epsilon: 0.01
SUCCESS: Error bound satisfied Eq. 2!


In [2]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-3

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    
    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.001)...
Internal SZ Ratio: 8.73
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 1.77 GB
Final Compression Ratio (rho): 8.7306
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.0849456787109375
Max Relative Error (calc): 0.0009998257737606764
Target Epsilon: 0.001
SUCCESS: Error bound satisfied Eq. 2!


In [5]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-4

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    
    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int64))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.0001)...
Internal SZ Ratio: 4.51
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 3.44 GB
Final Compression Ratio (rho): 4.5053
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.00848388671875
Max Relative Error (calc): 9.985685755964369e-05
Target Epsilon: 0.0001
SUCCESS: Error bound satisfied Eq. 2!



# Interpolation only algorithm



In [6]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.INTERP        # Interpolation only

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.01)...
Internal SZ Ratio: 36.12
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 0.43 GB
Final Compression Ratio (rho): 36.1225
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.8495941162109375
Max Relative Error (calc): 0.009999874047935009
Target Epsilon: 0.01
SUCCESS: Error bound satisfied Eq. 2!


In [7]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-3

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.INTERP        # Interpolation only

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.001)...
Internal SZ Ratio: 8.61
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 1.80 GB
Final Compression Ratio (rho): 8.6148
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.0849456787109375
Max Relative Error (calc): 0.0009998257737606764
Target Epsilon: 0.001
SUCCESS: Error bound satisfied Eq. 2!


In [10]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-4

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.INTERP        # Interpolation only

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int64))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.0001)...
Internal SZ Ratio: 4.48
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 3.46 GB
Final Compression Ratio (rho): 4.4792
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.00848388671875
Max Relative Error (calc): 9.985685755964369e-05
Target Epsilon: 0.0001
SUCCESS: Error bound satisfied Eq. 2!





# Lorenzo/regression


In [12]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.LORENZO_REG   # Lorenzo/regression

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.01)...
Internal SZ Ratio: 27.31
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 0.57 GB
Final Compression Ratio (rho): 27.3130
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.8495941162109375
Max Relative Error (calc): 0.009999874047935009
Target Epsilon: 0.01
SUCCESS: Error bound satisfied Eq. 2!


In [13]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-3

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.LORENZO_REG   # Lorenzo/regression

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.001)...
Internal SZ Ratio: 11.17
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 1.39 GB
Final Compression Ratio (rho): 11.1724
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.0849456787109375
Max Relative Error (calc): 0.0009998257737606764
Target Epsilon: 0.001
SUCCESS: Error bound satisfied Eq. 2!


In [14]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-4

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=855, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=1215, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    config.cmprAlgo = szAlgorithm.LORENZO_REG   # Lorenzo/regression

    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int64))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.0001)...
Internal SZ Ratio: 5.58
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.48 GB
Size of Array G (disk): 2.77 GB
Final Compression Ratio (rho): 5.5819
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.00848388671875
Max Relative Error (calc): 9.985685755964369e-05
Target Epsilon: 0.0001
SUCCESS: Error bound satisfied Eq. 2!


In [3]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG" 

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2 

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def main():
    print(f"Loading data from {INPUT_FILE}...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Please download the dataset[cite: 90].")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    print(f"Data Range: {v_range} (Min: {d_min}, Max: {d_max})")

    print("Storing original data D in TileDB...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=10, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=17, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=81, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(domain=dom_d, sparse=False, attrs=[tiledb.Attr(name="temp", dtype=DTYPE)])
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d

    print(f"Compressing with SZ3 (Relative Error = {EPSILON})...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON 
    
    compressed_bytes, ratio_sz_internal = sz.compress(data_d, config)
    
    print(f"Internal SZ Ratio: {ratio_sz_internal:.2f}")

    print("Storing compressed data G in TileDB...")
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)

    comp_size = compressed_bytes.size
    dom_g = tiledb.Domain(tiledb.Dim(name="index", domain=(0, comp_size-1), tile=comp_size, dtype=np.int32))
    schema_g = tiledb.ArraySchema(domain=dom_g, sparse=False, attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)])
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = compressed_bytes

    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    
    rho = size_D_folder / size_G_folder
    print("-" * 30)
    print(f"Size of Array D (disk): {size_D_folder / (1024**3):.2f} GB")
    print(f"Size of Array G (disk): {size_G_folder / (1024**3):.2f} GB")
    print(f"Final Compression Ratio (rho): {rho:.4f}")
    print("-" * 30)

    print("Verifying Decompression and Error Bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        read_bytes = A[:]['bytes']
    
    decompressed_data, dec_config = sz.decompress(read_bytes, DTYPE,  SHAPE)
    
    diff = np.abs(data_d - decompressed_data)
    max_pointwise_diff = diff.max()
    
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print(f"Max Absolute Error: {max_pointwise_diff}")
    print(f"Max Relative Error (calc): {actual_max_rel_error}")
    print(f"Target Epsilon: {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9: # small buffer for float precision
        print("SUCCESS: Error bound satisfied Eq. 2!")
    else:
        print("WARNING: Error bound NOT satisfied.")
        
if __name__ == "__main__":
    main()

Loading data from Redsea_t2_4k_gan.dat...
Data Range: 84.96047973632812 (Min: 225.58950805664062, Max: 310.54998779296875)
Storing original data D in TileDB...
Compressing with SZ3 (Relative Error = 0.01)...
Internal SZ Ratio: 39.77
Storing compressed data G in TileDB...
------------------------------
Size of Array D (disk): 15.71 GB
Size of Array G (disk): 0.39 GB
Final Compression Ratio (rho): 40.3494
------------------------------
Verifying Decompression and Error Bounds...
Max Absolute Error: 0.8495941162109375
Max Relative Error (calc): 0.009999874047935009
Target Epsilon: 0.01
SUCCESS: Error bound satisfied Eq. 2!


**EXPERIMENTS

In [6]:
import numpy as np
import os
from pysz import sz, szConfig, szErrorBoundMode

# Configuration
INPUT_FILE = "Redsea_t2_4k_gan.dat"
OUTPUT_COMPRESSED = "compressed_data.dat"
OUTPUT_DECOMPRESSED = "decompressed_data.dat"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

def get_file_size(filename):
    """Get file size in bytes"""
    if os.path.exists(filename):
        return os.path.getsize(filename)
    return 0

def format_size(size_bytes):
    """Format bytes to human readable"""
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.2f} TB"

print("=" * 70)
print("COMPRESSION TEST WITHOUT TileDB")
print("=" * 70)
print()

# Step 1: Load original data
print("📂 Step 1: Loading original data...")
try:
    data_original = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
except FileNotFoundError:
    print(f"   ❌ Error: {INPUT_FILE} not found!")
    exit(1)

original_size = data_original.nbytes
print(f"   📊 Size in memory: {format_size(original_size)}")

# Calculate data range
d_max = data_original.max()
d_min = data_original.min()
v_range = d_max - d_min
print(f"   📈 Data range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
print()

# Step 2: Compress with SZ3
print("🗜️  Step 2: Compressing with SZ3...")
config = szConfig()
config.errorBoundMode = szErrorBoundMode.REL
config.relErrorBound = EPSILON

compressed_bytes, sz_ratio = sz.compress(data_original, config)
print(f"   ✅ Compression complete!")
print(f"   📊 SZ3 internal ratio: {sz_ratio:.2f}×")
print()

# Step 3: Save compressed data to file
print("💾 Step 3: Saving compressed data...")
compressed_bytes.tofile(OUTPUT_COMPRESSED)
compressed_size = get_file_size(OUTPUT_COMPRESSED)
print(f"   ✅ Saved to: {OUTPUT_COMPRESSED}")
print(f"   📊 Compressed file size: {format_size(compressed_size)}")
print()

# # Step 4: Calculate compression ratio
# print("📊 Step 4: Calculating compression ratio...")
# compression_ratio = original_size / compressed_size
# print(f"   Original size:    {format_size(original_size)}")
# print(f"   Compressed size:  {format_size(compressed_size)}")
# print(f"   🎯 Compression ratio: {compression_ratio:.2f}×")
# print(f"   💾 Space saved: {((original_size - compressed_size) / original_size * 100):.1f}%")
# print()

# # Step 5: Decompress and verify
# print("🔓 Step 5: Decompressing and verifying...")
# compressed_read = np.fromfile(OUTPUT_COMPRESSED, dtype=np.uint8)
# data_decompressed, _ = sz.decompress(compressed_read, DTYPE, SHAPE)

# # Save decompressed for inspection
# data_decompressed.tofile(OUTPUT_DECOMPRESSED)
# decompressed_size = get_file_size(OUTPUT_DECOMPRESSED)
# print(f"   ✅ Decompressed successfully")
# print(f"   📊 Decompressed size: {format_size(decompressed_size)}")
# print()

# # Step 6: Verify error bounds
# print("🔍 Step 6: Verifying error bounds...")
# diff = np.abs(data_original - data_decompressed)
# max_abs_error = diff.max()
# max_rel_error = max_abs_error / v_range

# print(f"   Max absolute error: {max_abs_error:.6f}")
# print(f"   Max relative error: {max_rel_error:.6f}")
# print(f"   Target epsilon:     {EPSILON}")

# if max_rel_error <= EPSILON + 1e-9:
#     print(f"   ✅ SUCCESS: Error bound satisfied!")
# else:
#     print(f"   ❌ FAILED: Error bound exceeded!")
# print()

# # Step 7: Comparison summary
# print("=" * 70)
# print("📊 FINAL SUMMARY (NO TileDB)")
# print("=" * 70)
# print(f"Original file:      {format_size(original_size):>15}")
# print(f"Compressed file:    {format_size(compressed_size):>15}")
# print(f"Compression ratio:  {compression_ratio:>14.2f}×")
# print(f"Space saved:        {((original_size - compressed_size) / original_size * 100):>13.1f}%")
# print("=" * 70)
# print()

# # Step 8: What if we used TileDB?
# print("🤔 What happens with TileDB?")
# print("-" * 70)
# print("TileDB adds:")
# print("  • Schema files (~10-20 MB)")
# print("  • Metadata for each tile (~10-50 KB per tile)")
# print("  • Fragment info files (~5-10 MB)")
# print("  • Directory structure overhead")
# print()
# print("Estimated TileDB overhead:")

# # Rough estimates
# tile_count_d = 4000  # For arrayD with tile=(1,855,1215)
# tile_count_g = 1     # For arrayG with tile=comp_size

# overhead_d = 20 * 1024**2 + tile_count_d * 10 * 1024  # Schema + per-tile metadata
# overhead_g = 20 * 1024**2 + tile_count_g * 10 * 1024

# total_with_tiledb = original_size + overhead_d + compressed_size + overhead_g
# ratio_with_tiledb = original_size / (compressed_size + overhead_g)

# print(f"  arrayD overhead:    ~{format_size(overhead_d)}")
# print(f"  arrayG overhead:    ~{format_size(overhead_g)}")
# print(f"  Total overhead:     ~{format_size(overhead_d + overhead_g)}")
# print()
# print(f"Compression ratio with TileDB: ~{ratio_with_tiledb:.2f}×")
# print(f"Ratio WITHOUT TileDB:           {compression_ratio:.2f}×")
# print(f"Difference:                     {compression_ratio - ratio_with_tiledb:.2f}×")
# print("=" * 70)
# print()

# # Cleanup info
# print("📁 Files created:")
# print(f"  • {OUTPUT_COMPRESSED} ({format_size(compressed_size)})")
# print(f"  • {OUTPUT_DECOMPRESSED} ({format_size(decompressed_size)})")
# print()
# print("✨ Test complete! You can delete these files when done.")

COMPRESSION TEST WITHOUT TileDB

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   📊 Size in memory: 15.48 GB
   📈 Data range: 84.96 (min: 225.59, max: 310.55)

🗜️  Step 2: Compressing with SZ3...
   ✅ Compression complete!
   📊 SZ3 internal ratio: 39.77×

💾 Step 3: Saving compressed data...
   ✅ Saved to: compressed_data.dat
   📊 Compressed file size: 398.55 MB



In [1]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 4000   # Full time dimension
TILE_X = 1      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         4000 × 1 × 1215
   Tiles per dim:     1 × 855 × 1
   Total tiles:       855
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 855 tiles...
           1/855 tiles (  0.1%) - Compressed:     1.17 MB
          85/855 tiles (  9.9%) - Compressed:    95.26 MB
         170/855 tiles ( 19.9%) - Compressed:   194.68 MB
         255/855 tiles ( 29.8%) - Compressed:   302.96 MB
         340/855 tiles ( 39.8%) - Compressed:   413.43 MB
         425/855 tiles ( 49.7%) - Compressed:   534.71 MB
         510/855 tiles ( 59.6%) - Compressed:   666.12 MB
         595/855 tiles ( 69.6%) - Compressed:   806.70 MB
    

In [2]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 1   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         1 × 855 × 1215
   Tiles per dim:     4000 × 1 × 1
   Total tiles:       4000
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 4000 tiles...
           1/4000 tiles (  0.0%) - Compressed:     0.10 MB
         400/4000 tiles ( 10.0%) - Compressed:    39.14 MB
         800/4000 tiles ( 20.0%) - Compressed:    78.26 MB
        1200/4000 tiles ( 30.0%) - Compressed:   117.33 MB
        1600/4000 tiles ( 40.0%) - Compressed:   156.44 MB
        2000/4000 tiles ( 50.0%) - Compressed:   195.55 MB
        2400/4000 tiles ( 60.0%) - Compressed:   234.50 MB
        2800/4000 tiles ( 70.0%) - Compressed:   273.

In [3]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 100   # Full time dimension
TILE_X = 100      # One X position
TILE_Y = 100   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         100 × 100 × 100
   Tiles per dim:     40 × 9 × 13
   Total tiles:       4680
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 4680 tiles...
           1/4680 tiles (  0.0%) - Compressed:     0.27 MB
         468/4680 tiles ( 10.0%) - Compressed:   106.60 MB
         936/4680 tiles ( 20.0%) - Compressed:   213.58 MB
        1404/4680 tiles ( 30.0%) - Compressed:   320.39 MB
        1872/4680 tiles ( 40.0%) - Compressed:   425.82 MB
        2340/4680 tiles ( 50.0%) - Compressed:   533.08 MB
        2808/4680 tiles ( 60.0%) - Compressed:   639.48 MB
        3276/4680 tiles ( 70.0%) - Compressed:   746.

In [7]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 400   # Full time dimension
TILE_X = 400      # One X position
TILE_Y = 400   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    size_D_folder = get_folder_size(ARRAY_D_NAME)

    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")

    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         400 × 400 × 400
   Tiles per dim:     10 × 3 × 4
   Total tiles:       120
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored
Size of arrayD (disk):          28.62 GB

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 120 tiles...
           1/120 tiles (  0.8%) - Compressed:    12.43 MB
          12/120 tiles ( 10.0%) - Compressed:    66.69 MB
          24/120 tiles ( 20.0%) - Compressed:   132.25 MB
          36/120 tiles ( 30.0%) - Compressed:   199.43 MB
          48/120 tiles ( 40.0%) - Compressed:   266.15 MB
          60/120 tiles ( 50.0%) - Compressed:   332.32 MB
          72/120 tiles ( 60.0%) - Compressed:   398.93 MB
          84/120 tiles

In [5]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 400   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         400 × 855 × 1215
   Tiles per dim:     10 × 1 × 1
   Total tiles:       10
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 10 tiles...
           1/10 tiles ( 10.0%) - Compressed:    40.70 MB
           2/10 tiles ( 20.0%) - Compressed:    81.64 MB
           3/10 tiles ( 30.0%) - Compressed:   122.28 MB
           4/10 tiles ( 40.0%) - Compressed:   163.29 MB
           5/10 tiles ( 50.0%) - Compressed:   204.15 MB
           6/10 tiles ( 60.0%) - Compressed:   244.88 MB
           7/10 tiles ( 70.0%) - Compressed:   285.26 MB
           8/10 tiles ( 80.0%) - Compressed:   326.18 MB
           9/1

In [6]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 10   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         10 × 855 × 1215
   Tiles per dim:     400 × 1 × 1
   Total tiles:       400
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 400 tiles...
           1/400 tiles (  0.2%) - Compressed:     1.07 MB
          40/400 tiles ( 10.0%) - Compressed:    43.73 MB
          80/400 tiles ( 20.0%) - Compressed:    87.41 MB
         120/400 tiles ( 30.0%) - Compressed:   131.10 MB
         160/400 tiles ( 40.0%) - Compressed:   174.84 MB
         200/400 tiles ( 50.0%) - Compressed:   218.74 MB
         240/400 tiles ( 60.0%) - Compressed:   262.35 MB
         280/400 tiles ( 70.0%) - Compressed:   306.01 MB
    

In [1]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 1   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         1 × 855 × 1215
   Tiles per dim:     4000 × 1 × 1
   Total tiles:       4000
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 4000 tiles...
           1/4000 tiles (  0.0%) - Compressed:     0.10 MB
         400/4000 tiles ( 10.0%) - Compressed:    39.14 MB
         800/4000 tiles ( 20.0%) - Compressed:    78.26 MB
        1200/4000 tiles ( 30.0%) - Compressed:   117.33 MB
        1600/4000 tiles ( 40.0%) - Compressed:   156.44 MB
        2000/4000 tiles ( 50.0%) - Compressed:   195.55 MB
        2400/4000 tiles ( 60.0%) - Compressed:   234.50 MB
        2800/4000 tiles ( 70.0%) - Compressed:   273.

In [4]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 1   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    config.cmprAlgo = szAlgorithm.INTERP
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         1 × 855 × 1215
   Tiles per dim:     4000 × 1 × 1
   Total tiles:       4000
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 4000 tiles...
           1/4000 tiles (  0.0%) - Compressed:     0.11 MB
         400/4000 tiles ( 10.0%) - Compressed:    42.46 MB
         800/4000 tiles ( 20.0%) - Compressed:    84.91 MB
        1200/4000 tiles ( 30.0%) - Compressed:   127.29 MB
        1600/4000 tiles ( 40.0%) - Compressed:   169.71 MB
        2000/4000 tiles ( 50.0%) - Compressed:   212.12 MB
        2400/4000 tiles ( 60.0%) - Compressed:   254.37 MB
        2800/4000 tiles ( 70.0%) - Compressed:   296.

In [5]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode, szAlgorithm
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 1   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    config.cmprAlgo = szAlgorithm.LORENZO_REG
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         1 × 855 × 1215
   Tiles per dim:     4000 × 1 × 1
   Total tiles:       4000
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 4000 tiles...
           1/4000 tiles (  0.0%) - Compressed:     0.11 MB
         400/4000 tiles ( 10.0%) - Compressed:    43.94 MB
         800/4000 tiles ( 20.0%) - Compressed:    87.84 MB
        1200/4000 tiles ( 30.0%) - Compressed:   131.73 MB
        1600/4000 tiles ( 40.0%) - Compressed:   175.62 MB
        2000/4000 tiles ( 50.0%) - Compressed:   219.50 MB
        2400/4000 tiles ( 60.0%) - Compressed:   263.26 MB
        2800/4000 tiles ( 70.0%) - Compressed:   307.

In [8]:
import numpy as np
import tiledb
import os
import shutil
from pysz import sz, szConfig, szErrorBoundMode
import math

INPUT_FILE = "Redsea_t2_4k_gan.dat"
ARRAY_D_NAME = "arrayD" 
ARRAY_G_NAME = "arrayG"
ARRAY_SIZES_NAME = "arraySizes"

SHAPE = (4000, 855, 1215)
DTYPE = np.float32
EPSILON = 1e-2

# ============= CONFIGURABLE TILE SIZE =============
# Change these to test different tiling strategies!
TILE_T = 4000   # Full time dimension
TILE_X = 855      # One X position
TILE_Y = 1215   # Full Y dimension

# Alternative configurations to try:
# TILE_T, TILE_X, TILE_Y = 1, 855, 1215      # Original: 4000 tiles
# TILE_T, TILE_X, TILE_Y = 4000, 1, 1215     # This example: 855 tiles
# TILE_T, TILE_X, TILE_Y = 10, 100, 100      # Small chunks: 468000 tiles
# TILE_T, TILE_X, TILE_Y = 100, 100, 100     # Medium chunks
# ===================================================

def get_folder_size(folder_path):
    """Calculate total size of folder including all subfolders"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def calculate_tile_grid(shape, tile_size):
    """Calculate how many tiles needed for each dimension"""
    tiles_per_dim = []
    total_tiles = 1
    
    for i, (dim_size, tile_dim) in enumerate(zip(shape, tile_size)):
        num_tiles = math.ceil(dim_size / tile_dim)
        tiles_per_dim.append(num_tiles)
        total_tiles *= num_tiles
    
    return tiles_per_dim, total_tiles

def extract_tile(data, tile_idx, shape, tile_size, tiles_per_dim):
    """Extract a specific tile from 3D data"""
    # Convert flat index to 3D coordinates
    idx_t = tile_idx // (tiles_per_dim[1] * tiles_per_dim[2])
    idx_x = (tile_idx % (tiles_per_dim[1] * tiles_per_dim[2])) // tiles_per_dim[2]
    idx_y = tile_idx % tiles_per_dim[2]
    
    # Calculate start and end positions
    start_t = idx_t * tile_size[0]
    end_t = min(start_t + tile_size[0], shape[0])
    
    start_x = idx_x * tile_size[1]
    end_x = min(start_x + tile_size[1], shape[1])
    
    start_y = idx_y * tile_size[2]
    end_y = min(start_y + tile_size[2], shape[2])
    
    # Extract tile
    tile = data[start_t:end_t, start_x:end_x, start_y:end_y].copy()
    
    return tile, (start_t, end_t, start_x, end_x, start_y, end_y)

def insert_tile(data, tile, coords):
    """Insert a decompressed tile back into data"""
    start_t, end_t, start_x, end_x, start_y, end_y = coords
    data[start_t:end_t, start_x:end_x, start_y:end_y] = tile

def main():
    print("=" * 80)
    print("UNIVERSAL TILE-BASED COMPRESSION")
    print("=" * 80)
    print()
    
    # ============= CONFIGURATION INFO =============
    tiles_per_dim, total_tiles = calculate_tile_grid(SHAPE, (TILE_T, TILE_X, TILE_Y))
    
    print(f"📊 TILE CONFIGURATION")
    print(f"   Data shape:        {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]}")
    print(f"   Tile size:         {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"   Tiles per dim:     {tiles_per_dim[0]} × {tiles_per_dim[1]} × {tiles_per_dim[2]}")
    print(f"   Total tiles:       {total_tiles}")
    print(f"   Compression mode:  ε = {EPSILON} (relative error bound)")
    print()

    # ============= LOAD ORIGINAL DATA =============
    print("📂 Step 1: Loading original data...")
    try:
        data_d = np.fromfile(INPUT_FILE, dtype=DTYPE).reshape(SHAPE)
    except FileNotFoundError:
        print(f"   ❌ Error: {INPUT_FILE} not found!")
        return

    d_max = data_d.max()
    d_min = data_d.min()
    v_range = d_max - d_min
    
    print(f"   ✅ Loaded: {SHAPE[0]} × {SHAPE[1]} × {SHAPE[2]} elements")
    print(f"   Size: {data_d.nbytes / (1024**3):.2f} GB")
    print(f"   Range: {v_range:.2f} (min: {d_min:.2f}, max: {d_max:.2f})")
    print()

    # ============= STORE ORIGINAL DATA IN TileDB =============
    print("💾 Step 2: Storing original data in arrayD...")
    if os.path.exists(ARRAY_D_NAME):
        shutil.rmtree(ARRAY_D_NAME)
    
    dom_d = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, SHAPE[0]-1), tile=TILE_T, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, SHAPE[1]-1), tile=TILE_X, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, SHAPE[2]-1), tile=TILE_Y, dtype=np.int32)
    )
    schema_d = tiledb.ArraySchema(
        domain=dom_d,
        sparse=False,
        attrs=[tiledb.Attr(name="temp", dtype=DTYPE)]
    )
    tiledb.DenseArray.create(ARRAY_D_NAME, schema_d)
    
    with tiledb.DenseArray(ARRAY_D_NAME, mode='w') as A:
        A[:] = data_d
    
    print(f"   ✅ Original data stored")
    print()

    # ============= TILE-BASED COMPRESSION =============
    print(f"🗜️  Step 3: Tile-based compression with SZ3...")
    
    config = szConfig()
    config.errorBoundMode = szErrorBoundMode.REL
    config.relErrorBound = EPSILON
    
    all_compressed_tiles = []
    all_tile_sizes = []
    total_compressed_size = 0
    
    print(f"   Compressing {total_tiles} tiles...")
    
    for tile_idx in range(total_tiles):
        # Extract tile
        tile_data, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        
        # Compress tile
        compressed_tile, _ = sz.compress(tile_data, config)
        
        all_compressed_tiles.append(compressed_tile)
        tile_size = len(compressed_tile)
        all_tile_sizes.append(tile_size)
        total_compressed_size += tile_size
        
        # Progress indicator
        progress_interval = max(1, total_tiles // 10)  # Show progress 10 times
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / total_tiles) * 100
            print(f"      {tile_idx + 1:6d}/{total_tiles} tiles ({percentage:5.1f}%) - "
                  f"Compressed: {total_compressed_size / (1024**2):8.2f} MB")
    
    print(f"   ✅ Compression complete!")
    print(f"   Total compressed: {total_compressed_size / (1024**2):.2f} MB")
    print()

    # ============= STORE COMPRESSED DATA IN TileDB =============
    print("💾 Step 4: Storing compressed tiles in arrayG...")
    
    if os.path.exists(ARRAY_G_NAME):
        shutil.rmtree(ARRAY_G_NAME)
    
    # Concatenate all compressed tiles
    concatenated_bytes = np.concatenate(all_compressed_tiles, axis=0)
    
    # Store in TileDB
    dom_g = tiledb.Domain(
        tiledb.Dim(name="index", domain=(0, len(concatenated_bytes)-1), 
                   tile=len(concatenated_bytes), dtype=np.int32)
    )
    schema_g = tiledb.ArraySchema(
        domain=dom_g,
        sparse=False,
        attrs=[tiledb.Attr(name="bytes", dtype=np.uint8)]
    )
    tiledb.DenseArray.create(ARRAY_G_NAME, schema_g)
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='w') as A:
        A[:] = concatenated_bytes
    
    print(f"   ✅ Compressed data stored")
    print()

    # ============= STORE TILE SIZES METADATA =============
    print("💾 Step 5: Storing tile metadata (sizes)...")
    
    if os.path.exists(ARRAY_SIZES_NAME):
        shutil.rmtree(ARRAY_SIZES_NAME)
    
    tile_sizes_array = np.array(all_tile_sizes, dtype=np.int64)
    
    dom_sizes = tiledb.Domain(
        tiledb.Dim(name="tile_id", domain=(0, len(tile_sizes_array)-1), 
                   tile=len(tile_sizes_array), dtype=np.int32)
    )
    schema_sizes = tiledb.ArraySchema(
        domain=dom_sizes,
        sparse=False,
        attrs=[tiledb.Attr(name="size", dtype=np.int64)]
    )
    tiledb.DenseArray.create(ARRAY_SIZES_NAME, schema_sizes)
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='w') as A:
        A[:] = tile_sizes_array
    
    print(f"   ✅ Tile metadata stored ({len(tile_sizes_array)} tile sizes)")
    print()

    # ============= CALCULATE COMPRESSION RATIO =============
    print("📊 Step 6: Calculating compression ratio...")
    
    size_D_folder = get_folder_size(ARRAY_D_NAME)
    size_G_folder = get_folder_size(ARRAY_G_NAME)
    size_sizes_folder = get_folder_size(ARRAY_SIZES_NAME)
    
    total_stored_size = size_G_folder + size_sizes_folder
    rho = size_D_folder / total_stored_size
    
    print("-" * 80)
    print(f"Size of arrayD (disk):     {size_D_folder / (1024**3):10.2f} GB")
    print(f"Size of arrayG (disk):     {size_G_folder / (1024**3):10.2f} GB  (compressed tiles)")
    print(f"Size of arraySizes (disk): {size_sizes_folder / (1024**2):10.2f} MB  (tile metadata)")
    print(f"Total compression size:    {total_stored_size / (1024**3):10.2f} GB")
    print("-" * 80)
    print(f"🎯 Compression Ratio ρ:    {rho:10.4f}×")
    print("-" * 80)
    print()

    # ============= VERIFY ERROR BOUNDS =============
    print("🔍 Step 7: Verifying decompression and error bounds...")
    
    with tiledb.DenseArray(ARRAY_G_NAME, mode='r') as A:
        concatenated_read = A[:]['bytes']
    
    with tiledb.DenseArray(ARRAY_SIZES_NAME, mode='r') as A:
        tile_sizes_read = A[:]['size']
    
    # Decompress all tiles and reconstruct
    data_reconstructed = np.zeros(SHAPE, dtype=DTYPE)
    byte_offset = 0
    
    print(f"   Decompressing {len(tile_sizes_read)} tiles...")
    
    for tile_idx, tile_size_val in enumerate(tile_sizes_read):
        tile_size_val = int(tile_size_val)
        compressed_tile_bytes = concatenated_read[byte_offset:byte_offset + tile_size_val]
        byte_offset += tile_size_val
        
        # Get tile dimensions
        _, coords = extract_tile(data_d, tile_idx, SHAPE, (TILE_T, TILE_X, TILE_Y), tiles_per_dim)
        start_t, end_t, start_x, end_x, start_y, end_y = coords
        tile_shape = (end_t - start_t, end_x - start_x, end_y - start_y)
        
        # Decompress
        decompressed_tile, _ = sz.decompress(compressed_tile_bytes, DTYPE, tile_shape)
        
        # Insert back
        insert_tile(data_reconstructed, decompressed_tile, coords)
        
        progress_interval = max(1, len(tile_sizes_read) // 10)
        if (tile_idx + 1) % progress_interval == 0 or tile_idx == 0:
            percentage = ((tile_idx + 1) / len(tile_sizes_read)) * 100
            print(f"      {tile_idx + 1:6d}/{len(tile_sizes_read)} tiles ({percentage:5.1f}%)")
    
    # Verify error bounds
    diff = np.abs(data_d - data_reconstructed)
    max_pointwise_diff = diff.max()
    actual_max_rel_error = max_pointwise_diff / v_range
    
    print()
    print(f"Max Absolute Error:   {max_pointwise_diff:.8f}")
    print(f"Max Relative Error:   {actual_max_rel_error:.8f}")
    print(f"Target Epsilon:       {EPSILON}")
    
    if actual_max_rel_error <= EPSILON + 1e-9:
        print("✅ SUCCESS: Error bound satisfied (Eq. 3)!")
    else:
        print("❌ FAILED: Error bound NOT satisfied!")
    
    print()
    print("=" * 80)
    print("COMPRESSION SUMMARY")
    print("=" * 80)
    print(f"Tile configuration:    {TILE_T} × {TILE_X} × {TILE_Y}")
    print(f"Total tiles:           {total_tiles}")
    print(f"Compression ratio ρ:   {rho:.4f}×")
    print(f"Space saved:           {((size_D_folder - total_stored_size) / size_D_folder * 100):.1f}%")
    print(f"Error bound:           {actual_max_rel_error:.8f} ≤ {EPSILON}")
    print("=" * 80)

if __name__ == "__main__":
    main()

UNIVERSAL TILE-BASED COMPRESSION

📊 TILE CONFIGURATION
   Data shape:        4000 × 855 × 1215
   Tile size:         4000 × 855 × 1215
   Tiles per dim:     1 × 1 × 1
   Total tiles:       1
   Compression mode:  ε = 0.01 (relative error bound)

📂 Step 1: Loading original data...
   ✅ Loaded: 4000 × 855 × 1215 elements
   Size: 15.48 GB
   Range: 84.96 (min: 225.59, max: 310.55)

💾 Step 2: Storing original data in arrayD...
   ✅ Original data stored

🗜️  Step 3: Tile-based compression with SZ3...
   Compressing 1 tiles...
           1/1 tiles (100.0%) - Compressed:   398.55 MB
   ✅ Compression complete!
   Total compressed: 398.55 MB

💾 Step 4: Storing compressed tiles in arrayG...
   ✅ Compressed data stored

💾 Step 5: Storing tile metadata (sizes)...
   ✅ Tile metadata stored (1 tile sizes)

📊 Step 6: Calculating compression ratio...
--------------------------------------------------------------------------------
Size of arrayD (disk):          15.48 GB
Size of arrayG (disk):        